In [17]:
import sys
import operator
import requests
import json
import twitter
from watson_developer_cloud import PersonalityInsightsV2 as PersonalityInsights

In [18]:
def analyze(handle):
    #Twitter credentials, IBM credentials,
    twitter_consumer_key = 'Wx526MM90fEAbuEVkmRGJGKOv'
    twitter_consumer_secret = 'HTmVYSIOGmcyJdZyUNQBvtmhFi6po7r8xdO0GEXVp97a4BXRDK'
    twitter_access_token = '91267293-ufU0lIwKHzykkrz1wwN1YQPOzFZNK8G4tpSXpNHE5'
    twitter_access_secret = 'XXcNKdNV6WpdQfrmYhSGTOKQA9hn7T7xDTraLyKU5HUZF'
    pi_username = '2c577fe0-f0eb-4d77-aa77-7e702f14c2cb'
    pi_password = 'kHQLZCQhFFOV'
    #Create instances
    twitter_api = twitter.Api(consumer_key=twitter_consumer_key, consumer_secret=twitter_consumer_secret, access_token_key=twitter_access_token, access_token_secret=twitter_access_secret)
    personality_insights = PersonalityInsights(username=pi_username, password=pi_password)
    #Get Status
    statuses = twitter_api.GetUserTimeline(screen_name=handle, count=800, include_rts=False)
    text = ""
    for status in statuses:
        if (status.lang =='en'): #English tweets
            text += status.text.encode('utf-8')
    #Personality Insights results
    pi_result = personality_insights.profile(text)
    print pi_result
    return pi_result

In [19]:
def flatten(orig):
    data = {}
    for c in orig['tree']['children']:
        if 'children' in c:
            for c2 in c['children']:
                if 'children' in c2:
                    for c3 in c2['children']:
                        if 'children' in c3:
                            for c4 in c3['children']:
                                if (c4['category'] == 'personality'):
                                    data[c4['id']] = c4['percentage']
                                    if 'children' not in c3:
                                        if (c3['category'] == 'personality'):
                                                data[c3['id']] = c3['percentage']
    return data

In [20]:
def compare(dict1, dict2):
    compared_data = {}
    for keys in dict1:
        if dict1[keys] != dict2[keys]:
                compared_data[keys]=abs(dict1[keys] - dict2[keys])
    return compared_data

In [21]:
# celebrity_handle = "@joshwink1"
celebrity_handle = "@realDonaldTrump"
#user_handle = "@senorlelo"
user_handle = "@potus"

In [22]:
user_result = analyze(user_handle)
celebrity_result = analyze(celebrity_handle)

{u'warnings': [], u'tree': {u'children': [{u'children': [{u'category': u'personality', u'percentage': 0.9860577436357568, u'children': [{u'category': u'personality', u'name': u'Openness', u'sampling_error': 0.05329525984, u'id': u'Openness', u'percentage': 0.7705284550747172, u'children': [{u'category': u'personality', u'percentage': 0.9499377146692803, u'id': u'Adventurousness', u'sampling_error': 0.04647429712, u'name': u'Adventurousness'}, {u'category': u'personality', u'percentage': 0.7766118318743385, u'id': u'Artistic interests', u'sampling_error': 0.09501483352000001, u'name': u'Artistic interests'}, {u'category': u'personality', u'percentage': 0.5358642853216583, u'id': u'Emotionality', u'sampling_error': 0.04426119888, u'name': u'Emotionality'}, {u'category': u'personality', u'percentage': 0.08516428722233343, u'id': u'Imagination', u'sampling_error': 0.057701593440000004, u'name': u'Imagination'}, {u'category': u'personality', u'percentage': 0.9645603266362897, u'id': u'Intel

In [23]:
#First, flatten the results from the Watson PI API
user = flatten(user_result)
celebrity = flatten(celebrity_result)
#Then, compare the results of the Watson PI API by calculating the distance between traits
compared_results = compare(user,celebrity)

In [24]:
sorted_result = sorted(compared_results.items(), key=operator.itemgetter(1))
for keys, value in sorted_result[:5]:
    print keys,
    print(user[keys]),
    print ('->'),
    print (celebrity[keys]),
    print ('->'),
    print (compared_results[keys])

Depression 0.0312291659392 -> 0.0286376659488 -> 0.00259149999042
Self-efficacy 0.918445358827 -> 0.915434695053 -> 0.00301066377371
Sympathy 0.995762367876 -> 0.992002771998 -> 0.00375959587807
Vulnerability 0.0149615494014 -> 0.0191664506473 -> 0.00420490124591
Self-discipline 0.968033299561 -> 0.974129454712 -> 0.00609615515133
